<a href="https://colab.research.google.com/github/fezilemahlangu/Reinforcement-Learning-Project/blob/master/MCTS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Installing Dependencies

In [ ]:
#https://towardsdatascience.com/deep-reinforcement-learning-and-monte-carlo-tree-search-with-connect-4-ba22a4713e7a

#https://ai-boson.github.io/mcts/


In [2]:
!apt update
!apt install -y cmake
!apt-get install -y build-essential autoconf libtool pkg-config
!apt-get install flex bison libbz2-dev
!pip install nle
!pip install minihack
# !python -m minihack.scripts.env_list
!pip install gym[atari,accept-rom-license]

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:9 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:10 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease [15.9 kB]
Get:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [83.3 kB]
Get:12 http://security.ubuntu.com/ubuntu bionic-security/restricted amd64 Packages [1,217 kB]
Hit:13 http://

In [7]:
!pip install gym[box2d]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 18.3 MB 76 kB/s 
     |████████████████████████████████| 2.1 MB 71.1 MB/s 
     |████████████████████████████████| 1.8 MB 60.0 MB/s 


# Libraries

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import gym
import nle
import minihack
from gym import spaces
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import Adam
import torch
import random
from minihack import RewardManager
from collections import defaultdict
from minihack import reward_manager
# import numpy as np
# from minihack import RewardManager
# from gym import spaces
from nle import nethack
import warnings
warnings.filterwarnings("ignore")

In [42]:
# import gym
# import minihack
# from minihack import reward_manager
# import numpy as np
# from minihack import RewardManager
# from gym import spaces
# from nle import nethack
# from numpy.lib.function_base import select

# class BasicWrapper(gym.Wrapper):
#     def __init__(self, env, seed=0, maxSteps = 10000):
#         super().__init__(env)
#         self.env = env
#         self.seedCustom = seed
#         self.maxSteps = maxSteps
#         self.augmentedReward = 0
#         self.seenStates = None
#         self.currStep = 0
#         self.repeatNum = 0
#         # self.oldest = None
#         # self.secOldest = None
#         # self.secNewest= None
#         # self.newest = None
#         # print("NOTE: in here the max steps are set in the construction - change this as desired but keep in mind many steps are needed since a lot of exploring is needed")
#         # print("NOTE: Changing the seed if done, must be done here in initialising the class")
#         # print("IFF this is done, he seed used in the custom gym will be different and hence this change must be accounted for there as well")
#         # print("The default seed will be set to 0 in all cases as a standard seed - which allows for us to compare results")
#         # print("hence changing the seed is not advisable \n environment has been initialised")
#     def fullReset(self):
#         self.seenStates = None
#         self.currStep = 0
#         self.repeatNum = 0


#     def reset(self):
#         self.env.seed(self.seedCustom)
#         state = self.env.reset()
#         state = self.selectObs(state)
#         return state

#     def lookThroughDict(self, temp):
#         solidStone = [73,116,39,115,32,115,111,108,105,100,32,115,116,111,110,101,46]
#                     #   73 116 39 115 32 115 111 108 105 100 32 115 116 111 110 101 46 
#         whatStrange = [87,104 ,97, 116, 32, 97, 32, 115, 116, 114, 97, 110, 103, 101, 32, 100, 105, 114, 101, 99, 116 ,105 ,111 ,110 ,33 ,32 ,32 ,78 ,101 ,118  ] # ,101 ,114 ,32 ,109 ,105 ,110 ,100 ,46
#         nothingZap = [89, 111, 117, 32 ,100 ,111 ,110 ,39 ,116, 32, 104, 97, 118, 101, 32, 97 ,110, 121, 116, 104, 105, 110, 103, 32] #  ,116, 111, 32, 122, 97, 112, 46 
#         whatDir = [73, 110, 32 ,119 ,104 ,97 ,116 ,32 ,100 ,105 ,114 ,101 ,99] # ,116 ,105 ,111 ,110 ,63 
#         noDoor = [89, 111, 117, 32, 115, 101, 101, 32 ,110 ,111 ,32 ,100 ,111 ,111 ,114 ,32 ,116] # ,101 ,114 ,101 ,46  at the end missing here
#         cnt = 0 
#         punishmentReward = 0.2
#         isMessage = False
#         solid = True
#         for char in solidStone:
#             if temp[cnt] != char:
#                 solid = False
#                 break
                
#                 # break
#             cnt += 1
#         if solid:
#             self.augmentedReward -= punishmentReward
#             return
#         strange = True
#         for char in whatStrange:
#             if temp[cnt] != char:
#                 strange = False
#                 # self.augmentedReward -= 0.3
#                 # return
#                 break
#             cnt += 1
#         if strange:
#             self.augmentedReward -= punishmentReward
#             return

#         zap = True
#         for char in nothingZap:
#             if temp[cnt] != char:
#                 strange = False
#                 # self.augmentedReward -= 0.3
#                 # return
#                 break
#             cnt += 1
#         if zap:
#             self.augmentedReward -= punishmentReward
#             return
#         whatDirec = True
#         for char in whatDir:
#             if temp[cnt] != char:
#                 whatDirec = False
#                 # self.augmentedReward -= 0.3
#                 # return
#                 break
#             cnt += 1
#         if whatDirec:
#             self.augmentedReward -= punishmentReward
#             return
#         noDoorHere = True
#         for char in noDoor:
#             if temp[cnt] != char:
#                 noDoorHere = False
#                 # self.augmentedReward -= 0.3
#                 # return
#                 break
#             cnt += 1
#         if noDoorHere:
#             self.augmentedReward -= punishmentReward
#             return
#         else:
#             self.augmentedReward += 0.01
#             return

#     def rewardExploration(self, temp):
#         seenBefore = False
        
#         # for state in self.seenStates:
#         #     if np.all(state == temp):
#         #         self.repeatNum += 1
#         #         self.augmentedReward -= 0.0025 * self.repeatNum
#         #         # seenBefore = True
#         #         return
#         # self.augmentedReward += 5
        
        

#     def selectObs(self, obs, desired=["chars","message","inv_letters"]):
#         tempState = np.array(())
#         for desire in desired:
#             temp = obs[desire]
#             if desire == "message":
#                 self.lookThroughDict(temp)
#             temp = np.array(temp)
#             temp = temp.astype(int)
#             temp = temp.flatten()
#             tempState = np.append(tempState, temp)
#         return tempState

#     def step(self, action, maxLength=10000):
#         self.currStep += 1
#         self.augmentedReward = 0
#         next_state, reward, done, info = self.env.step(action)
#         # if np.any(self.seenStates != None):
#         #     self.rewardExploration(next_state["chars"])
#         #     self.seenStates.append(next_state["chars"])
#         next_state = self.selectObs(next_state)
#         self.lookThroughDict(next_state)
#         # if np.any(self.seenStates != None):
#         #     self.rewardExploration(next_state)
#         #     self.seenStates.append(next_state)
#         # else:
#         #     self.seenStates = [next_state]
#         reward += self.augmentedReward
#         # if done == True:
#         #     if reward != 2:
#         #         reward = -0.0105*(self.maxSteps-self.currStep) - 0.0105*(self.maxSteps) 
#         #     elif reward == 2: 
#         #         reward = 50
#         # elif reward != -0.01 and reward != 20 and reward != 40:
#         #     reward -= 0.02
#         # if reward == 20:
#         #     print("it equipped the wand")
#         # if reward == 40:
#         #     print("it killed the minotaur")
#         # if np.any(self.seenStates == None):
#         #     self.seenStates = [[next_state]]
#         # else:
#         # self.augmentedReward = 0
#         # self.oldest = None
#         # self.secOldest = None
#         # self.secNewest= None
#         # self.newest = None
#         # if np.all(self.newest == None):
#         #     self.newest = next_state
#         # elif np.all(self.newest != None):
#         #     if np.all(self.secNewest == None):
#         #         self.secNewest = self.newest
#         #         self.newest = next_state
#         #     else:
#         #         if np.all(self.secOldest == None):
#         #             self.secOldest = self.secNewest
#         #             self.secNewest = self.newest
#         #             self.newest = next_state
#         #         else:
#         #             if np.all(next_state == self.secOldest):
#         #                 reward -= 0.02
#         #             if np.all(next_state == self.secNewest):
#         #                 reward -= 0.02
#         #             if np.all(next_state == self.newest):
#         #                 reward -= 0.02
#         return next_state, reward, done, info

# def createActionSpace():
#     moves = tuple(nethack.CompassDirection)
#     navActions = moves + (
#         # nethack.Command.APPLY,
#         # nethack.Command.AUTOPICKUP,
#         # nethack.Command.CAST,
#         # nethack.Command.CLOSE,
#         # nethack.Command.DROP,
#         nethack.Command.EAT,
#         # nethack.Command.ESC,
#         nethack.Command.FIRE,
#         nethack.Command.FIGHT,
#         # nethack.Command.INVOKE,
#         nethack.Command.KICK,
#         # nethack.Command.LOOK, 
#         # nethack.Command.LOOT,
#         # nethack.Command.OPEN,
#         # nethack.Command.PRAY,
#         # nethack.Command.PUTON,
#         # nethack.Command.QUAFF,
#         # nethack.Command.READ,
#         # nethack.Command.REMOVE,
#         # nethack.Command.RIDE,
#         # nethack.Command.RUB,
#         # nethack.Command.SEARCH,
#         # nethack.Command.TAKEOFF,
#         # nethack.Command.TAKEOFFALL,
#         # nethack.Command.THROW,
#         # nethack.Command.TIP,
#         # nethack.Command.WEAR,
#         nethack.Command.WIELD,
#         # nethack.Command.ZAP,
#     )
#     return navActions

# def customGym(maxLength=10000, seed=0):
#     reward_gen = RewardManager()
#     reward_gen.add_eat_event("apple", reward=1, repeatable=False)
#     reward_gen.add_wield_event("wand", reward=20, repeatable=False) # changed to convince the agent finding the wand and using it is good
#     reward_gen.add_location_event("sink", reward=-1, terminal_required=False)
#     reward_gen.add_kill_event("minotaur",reward=100, repeatable=False) #minotaur guards the exit and is in the maze, which requires the WoD to do
    
#     # may need more rewards as we continue
#     env = gym.make(
#         "MiniHack-Quest-Hard-v0",
#         observation_keys=("chars", "inv_letters", "message"),
#             reward_manager = reward_gen,
#             savedir = "./games",
#             actions=createActionSpace()
#     )
#     # print("Environment created")
#     # print("maxLength here represents the maxSteps possible for the agent, please ensure that the number given here is the same\n as the number given to the BasicWrapper otherwise there is a disconnect,\n by default their values if left unchanged and default then they are the same.")
#     env._max_episode_steps = maxLength
#     env.seed(seed)
#     return env


In [15]:
def makeEnv():
    global hyper_params
    MOVE_ACTIONS = tuple(nethack.CompassDirection)
    NAVIGATE_ACTIONS = MOVE_ACTIONS + (
        nethack.Command.OPEN,
        nethack.Command.SEARCH,
        nethack.Command.LOOK, 
        nethack.Command.JUMP, 
        nethack.Command.PICKUP,
        nethack.Command.WIELD, 
        nethack.Command.SWAP,
        nethack.Command.EAT,
        nethack.Command.ZAP,
        nethack.Command.LOOT,
        nethack.Command.PUTON,
        nethack.Command.APPLY,
        nethack.Command.CAST,
        nethack.Command.DIP,
        nethack.Command.READ,
        nethack.Command.INVOKE,
        nethack.Command.RUSH,
        nethack.Command.WEAR,
        nethack.Command.ENHANCE

        # Might need more? All actions and descriptions found here
        # https://minihack.readthedocs.io/en/latest/getting-started/action_spaces.html
    )
    reward_gen = RewardManager()
    reward_gen.add_kill_event("minotaur", reward=10)
    reward_gen.add_eat_event("apple", reward=1, repeatable=False)
    reward_gen.add_kill_event("goblin", reward=1)
    reward_gen.add_kill_event("jackal", reward=1)
    reward_gen.add_kill_event("giant rat", reward=1)
    reward_gen.add_location_event("sink", reward=-1, terminal_required=False)
    strings = list()
    strings.append("The door opens.")
    reward_gen.add_message_event(strings, reward=1)
    # Create env with modified actions
    # Probably can limit the observations as well
    pixel_obs = "pixel_crop"
    env = gym.make(
        hyper_params["env-name"],
        observation_keys=("chars", "inv_letters", "message"),
        actions=NAVIGATE_ACTIONS,
        reward_lose=-10,
        reward_win=10,
        penalty_step = -0.002,
        penalty_time = 0.002,
        reward_manager=reward_gen,
        savedir = "./games",
        max_episode_steps = hyper_params['num-steps']
    )
    env.seed(hyper_params["seed"])
    # env = RenderRGB(env, pixel_obs)
    # env = gym.wrappers.Monitor(env, "recordings", force=True)
    return env


# MCTS

In [20]:
#reward shaping 

# reward_manager = RewardManager()
# reward_manager.add_eat_event("apple", reward=1)
# reward_manager.add_wield_event("dagger", reward=2)
# reward_manager.add_wield_event("wand", reward=20)
# reward_manager.add_kill_event("minotaur",reward=40)
# reward_manager.add_location_event("sink", reward=-1, terminal_required=False)

# env = gym.make("MiniHack-Quest-Easy-v0",reward_manager=reward_manager)
hyper_params = {
        "seed": 42,  # which seed to use
        "env-name": "MiniHack-Quest-Hard-v0",  # name of the game
        "discount-factor": 0.99,  # discount factor
        "num-steps": int(5000),  # total number of steps to run the environment for
        "save-freq": 500, # number of iterations between each model save
    }
# env = BasicWrapper(customGym()) 
env = makeEnv()

In [17]:
env.action_space.n

27

# Node

In [23]:
class State():

  def __init__(self, state, reward=0,done=False,info=None):
        # print('init state')
        self.state_s = state
        self.reward = reward
        self.done = done
        self.info = info
        if done:
            self.legal_actions = []
        else:
            self.legal_actions = list(np.arange(env.action_space.n))

class MCTS():
  '''
  Class for MCTS node 
  '''
  def __init__(self,state, parent=None,  parent_action=None):

    self.state = state
    self.parent = parent # parent node in MCTS
    self.parent_action = parent_action # action that the parent took
    self.children = [] #children of parent node 
    self.visit_count = 0 # keeps count of how many times node has been visited
    self.rewards = defaultdict(int)
    self.unexplored_actions = self.state.legal_actions

  def expand(self,env):

    '''
    expand node and take unexplored action 
    '''
    action = self.unexplored_actions.pop()

    obs, reward, done, _ = env.step(action)

    obs = State(obs,reward,done,_)

    child_node = MCTS(obs,self,action) #obs should be state 

    self.children.append(child_node)

    return child_node, env

  def back_propagate(self,reward):
    '''
    Back propagate reward on all nodes from leaf to root and update visit count 
    '''
    
    self.visit_count += 1
    self.rewards[reward] += 1

    if self.parent:
      self.parent.back_propagate(reward)


  def rollout_policy(self, moves):
    #randomly selects a child node 
    '''
    rollout policy 
    '''

    return moves[np.random.randint(len(moves))]

  
  def rollout(self,env):
    '''
    rollout: until we reach the leaf node, randomly choose an action at each step and simulate 
    this act to receive an ave reward until done (ep is over)
    '''
    curr = self.state

    while not curr.done:
      moves = curr.legal_actions

      act = self.rollout_policy(moves)

      obs, reward, done, _ = env.step(act)

      curr = State(obs,reward,done,_)
    
    return curr.reward

  def tree_policy(self,env): #->fix 
    '''
    keeps expanding tree until terminal node is reached 
    '''
    # curr = self
    # while not curr.state.done:
    #   if len(self.children) < env.action_space.n:
    #     return self.expand(env)
    #   else:
    #     curr = curr.best_child(c_param = 0.1)

    # return curr, env

    current_node = self
    while not current_node.state.done:

        if len(current_node.children)==0:
            return current_node.expand(env)
        elif random.uniform(0,1)<.5:
            current_node = current_node.best_child()
        else:
            if not len(current_node.unexplored_actions) == 0:
                return current_node.expand(env)
            else:
                current_node = current_node.best_child()
    return current_node , env

  
  
  def n(self):
    return self.visit_count

  def q(self):
    # wins = self._results[1]
    # loses = self._results[-1]
    # return wins - loses

    total = 0
    for r in self.rewards:
        total += r*self.rewards[r]

    return total

  def best_child(self, c_param=0.1):
    
    # choices_weights = [(c.q() / c.n()) + c_param * np.sqrt((2 * np.log(self.n()) / c.n())) for c in self.children]
    # print(choices_weights)
    # return self.children[np.argmax(choices_weights)]

    choices_weights = [(c.q() / c.n()) + c_param * np.sqrt((2 * np.log(self.n()) / c.n())) for c in self.children]
    child = np.random.choice(np.flatnonzero(choices_weights == max(choices_weights)))
    return self.children[child]

  def best_action(self,actions):
    simulation_no = 70
	
	
    for i in range(simulation_no):
      new_env = env = makeEnv()
      # new_env = BasicWrapper(customGym()) 
      new_init_state = new_env.reset()

      for a in actions:
        obs,reward,done,_ = new_env.step(a)

      v = self.tree_policy(new_env)
      reward = v[0].rollout(v[1])
      v[0].back_propagate(reward)
	
    return self.best_child(c_param=0.1)

  
  # def best_child(self,c_p):
  #   '''
  #   finds best child using UCT
  #   xi is ave reward/value of all nodes beneath this node 
  #   ns is the numer of times the parent has been visited
  #   ni is the numer of times the child node i has been visited
  #   '''

  #   ns = self.visit_count #visit count of parent

  #   ni = [c.visit_count for c in self.children] #visit count of chilren 

  #   q = 0
  #   for r in self.rewards:
  #     q += r*self.rewards[r]

  #   first_term = q / ni

  #   second_term = c_p * np.sqrt((2*np.log(ns))/ni) 

  #   UCB1 = first_term + second_term 

  #   best_child = np.argmax(UCB1)

  #   return self.children[best_child]

  # def best_action(self):
  #   '''
  #   find next best action 
  #   '''

    
    



# Main

In [24]:
def main():

    iters =1000

    for mpx in range(1):
        

        prev_actions = []
        rewards = []
        actionsTaken = []
        
        env = env = makeEnv()
        # env = BasicWrapper(customGym()) 
        
        rewardArr = []
        
        initial_state = env.reset()
        initial_state = State(state = initial_state)
        # env.render()

        root = MCTS(state = initial_state)
     
        selected_node = root.best_action(prev_actions)
       
        prev_actions.append(selected_node.parent_action)


        next_state, reward, done, info = env.step(selected_node.parent_action)

        # rewards.append(reward)
        next_state = State(next_state,reward,done,info)

    
        cnt = 0
        
        # loops through the rest of the game repeatedly calling the 
        # best_action function and making the selected move in the environment
        for it in range(iters):
            if done:
                print("im done")
                break;
            cnt += 1
            print("Current Game {0}, current step {1}".format(mpx, cnt))
            selected_node = MCTS(state = next_state)
            selected_node = selected_node.best_action(prev_actions)


            prev_actions.append(selected_node.parent_action)

            next_state, reward, done, info = env.step(selected_node.parent_action)
            rewards.append(reward)
            print("reward: {}".format(reward))

            next_state = State(next_state,reward,done,info)

        rewardArr = np.array(rewards)
        # saves the rewards received during game
        # np.savetxt("rewardArr-{0}".format(mpx), rewardArr)

if __name__ == "__main__":
  main()

Current Game 0, current step 1
reward: -0.002
Current Game 0, current step 2
reward: -0.002
Current Game 0, current step 3
reward: -0.002
Current Game 0, current step 4
reward: -0.002
Current Game 0, current step 5
reward: -0.002
Current Game 0, current step 6
reward: -0.002
Current Game 0, current step 7
reward: -0.002
Current Game 0, current step 8
reward: -0.002
Current Game 0, current step 9
reward: -0.002
Current Game 0, current step 10
reward: -0.002
Current Game 0, current step 11
reward: -0.002
Current Game 0, current step 12
reward: -0.002
Current Game 0, current step 13
reward: -0.002
Current Game 0, current step 14
reward: -0.002
Current Game 0, current step 15
reward: -0.002
Current Game 0, current step 16
reward: -0.002
Current Game 0, current step 17


KeyboardInterrupt: ignored

In [46]:
print(1.900000000000000022e-01)
# 1.600000000000000311e-01
# 1.600000000000000311e-01
# 1.900000000000000022e-01

0.19
